In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Step 1: Load the CSV files
traffic_file_path = "traffic_202411_update.csv"
location_file_path = "location.csv" 

traffic_df = pd.read_csv(traffic_file_path)
location_df = pd.read_csv(location_file_path)

# Step 2: Extract unique LINK_NAME, BOROUGH, and LINK_POINTS
unique_traffic = traffic_df[['LINK_ID', 'LINK_POINTS', 'BOROUGH', 'LINK_NAME']].drop_duplicates()

# Step 3: Extract the first coordinate (latitude and longitude) from LINK_POINTS
unique_traffic['First_Coordinate'] = unique_traffic['LINK_POINTS'].apply(lambda x: x.split()[0])
unique_traffic[['Latitude', 'Longitude']] = unique_traffic['First_Coordinate'].str.split(',', expand=True)
unique_traffic['Latitude'] = unique_traffic['Latitude'].astype(float)
unique_traffic['Longitude'] = unique_traffic['Longitude'].astype(float)

# Step 4: Convert traffic and location data into GeoDataFrames
traffic_gdf = gpd.GeoDataFrame(
    unique_traffic,
    geometry=[Point(xy) for xy in zip(unique_traffic['Longitude'], unique_traffic['Latitude'])],
    crs="EPSG:4326"
)

location_gdf = gpd.GeoDataFrame(
    location_df,
    geometry=[Point(xy) for xy in zip(location_df['Longitude'], location_df['Latitude'])],
    crs="EPSG:4326"
)

# Step 5: Perform spatial join to find the closest LINK_NAME for each SiteName
matched_data = gpd.sjoin_nearest(location_gdf, traffic_gdf, how="left", distance_col="distance")

# Step 6: Prepare the final dataset
output_df = matched_data[['SiteID', 'SiteName', 'Latitude_left', 'Longitude_left', 'Address', 
                          'BOROUGH', 'LINK_NAME', 'LINK_ID', 'distance']]

# Step 7: Limit to 15 rows and save the result to a CSV file
output_df = output_df.nsmallest(50, 'distance')  # Closest 15 matches
output_path = "closest_links_to_sites.csv"
output_df.to_csv(output_path, index=False)

print(f"Closest links to sites saved to: {output_path}")


Closest links to sites saved to: closest_links_to_sites.csv


/Users/Ian/opt/anaconda3/lib/python3.9/site-packages/geopandas/array.py:365: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [12]:
import pandas as pd

traffic_file_path = "traffic_202411_update.csv"
traffic_df = pd.read_csv(traffic_file_path)
traffic_df['DATA_AS_OF'] = pd.to_datetime(traffic_df['DATA_AS_OF'])
traffic_df['DATA_AS_OF'] = traffic_df['DATA_AS_OF'].dt.floor('H')

aggregated_df = traffic_df.groupby(['DATA_AS_OF', 'LINK_ID', 'LINK_POINTS', 'BOROUGH', 'LINK_NAME']).agg(
    {'SPEED': 'mean', 'TRAVEL_TIME': 'mean'}
).reset_index()

aggregated_df['DATA_AS_OF'] = aggregated_df['DATA_AS_OF'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

output_path = "aggregated_traffic_data.csv"
aggregated_df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd

# Load the CSV file
file_path = "final_joined_data.csv" 
df = pd.read_csv(file_path)

# Drop the 'ID' and 'Borough2' columns
df = df.drop(columns=['ID', 'Borough2'])

# Parse decimal values to the correct format (ensure columns are floats)
df['AirQualityIndex'] = df['AirQualityIndex'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
df['TrafficSpeed'] = df['TrafficSpeed'].astype(float)

# Convert 'ObservationTime' to proper datetime format
df['ObservationTime'] = pd.to_datetime(df['ObservationTime'], format='%Y-%m-%d %H:%M')

# Save the cleaned data back to a new CSV file
output_file_path = "final_cleaned_data.csv"  # Replace with desired output file path
df.to_csv(output_file_path, index=False)

print("Data cleaning completed and saved to:", output_file_path)


Data cleaning completed and saved to: final_cleaned_data.csv


In [20]:
import pandas as pd

# Load the datasets (replace with actual file paths or dataframes)
aggregated_traffic_data = pd.read_csv("aggregated_traffic_data.csv")
closest_links_to_sites = pd.read_csv("closest_links_to_sites.csv")
air_quality_data = pd.read_csv("datasets_csv/AirQuality202411.csv")

air_quality_data['Time'] = pd.to_datetime(air_quality_data['Time'])
air_quality_data['Time'] = air_quality_data['Time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')


filtered_traffic_data = aggregated_traffic_data[aggregated_traffic_data['LINK_ID'].isin(closest_links_to_sites['LINK_ID'])]

merged_data = closest_links_to_sites.merge(
    air_quality_data,
    left_on='SiteID',
    right_on='SiteID',
    how='inner'
)

final_data = filtered_traffic_data.merge(
    merged_data,
    left_on=['LINK_ID', 'DATA_AS_OF'],
    right_on=['LINK_ID', 'Time'],
    how='inner'
)

# DATA_AS_OF,LINK_ID,LINK_POINTS,BOROUGH_x,LINK_NAME_x,SPEED,TRAVEL_TIME,SiteID,SiteName,Latitude_left,Longitude_left,Address,BOROUGH_y,LINK_NAME_y,distance,ID,Time,Air Quality Index

final_data = final_data[['Time', 'LINK_ID', 'BOROUGH_x', 'LINK_NAME_x', 'SPEED', 'SiteName', 'Latitude_left', 'Longitude_left', 'Air Quality Index']]
# Save the resulting dataset
output_path = "report_final_joined_data.csv"
final_data.to_csv(output_path, index=False)

final_data

,Time,LINK_ID,BOROUGH_x,LINK_NAME_x,SPEED,SiteName,Latitude_left,Longitude_left,Air Quality Index
0,2024-11-01 00:00:00.000000,4329473,Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,29.200000,Midtown-DOT,40.755082,-73.990415,14.47
1,2024-11-01 00:00:00.000000,4456452,Manhattan,TBB W - FDR S MANHATTAN TRUSS - E116TH STREET,32.925833,Mott Haven,40.806486,-73.922487,10.94
2,2024-11-01 00:00:00.000000,4616195,Staten Island,SIE E RICHMOND AVENUE - WOOLEY AVENUE,50.428333,SI Expwy,40.609209,-74.151182,6.83
3,2024-11-01 00:00:00.000000,4616298,Queens,VWE N MP4.63 (Exit 6 - Jamaica Ave) - MP6.39 (...,17.807500,Queens College,40.737107,-73.821556,11.33
4,2024-11-01 00:00:00.000000,4616309,Bronx,CBE E AMSTERDAM AVE(U/LVL) - MORRIS AVE,39.866667,Hamilton Bridge,40.846544,-73.933023,8.32
...,...,...,...,...,...,...,...,...,...
6666,2024-11-30 23:00:00.000000,4616328,Manhattan,FDR N Catherine Slip - 25th St,28.917273,FDR,40.722288,-73.974651,6.62
6667,2024-11-30 23:00:00.000000,4616328,Manhattan,FDR N Catherine Slip - 25th St,28.917273,BQE,40.702798,-73.960824,5.16
6668,2024-11-30 23:00:00.000000,4616332,Manhattan,FDR S 63rd - 25th St,39.195000,Queensboro Bridge,40.761234,-73.963886,6.38
6669,2024-11-30 23:00:00.000000,4616341,Manhattan,FDR S Catherine Slip - BKN Bridge Manhattan Side,25.316667,Manhattan Bridge,40.716510,-73.997004,7.30


In [16]:
report_final_joined_data = pd.read_csv("report_final_joined_data.csv")
report_final_joined_data

,DATA_AS_OF,LINK_ID,LINK_POINTS,BOROUGH_x,LINK_NAME_x,SPEED,TRAVEL_TIME,SiteID,SiteName,Latitude_left,Longitude_left,Address,BOROUGH_y,LINK_NAME_y,distance,ID,Time,Air Quality Index
0,2024-11-01 00:00:00.000000,4329473,"40.7578106,-73.996801 40.7604506,-74.003221 40...",Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,29.20,191.000000,36061NY09929,Midtown-DOT,40.755082,-73.990415,N Side of W 39th between 8th and 7th Ave,Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,0.006945,641594,2024-11-01 00:00:00.000,14.47
1,2024-11-01 00:00:00.000000,4329473,"40.7578106,-73.996801 40.7604506,-74.003221 40...",Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,29.20,191.000000,36061NY09929,Midtown-DOT,40.755082,-73.990415,N Side of W 39th between 8th and 7th Ave,Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,0.006945,641605,2024-11-01 01:00:00.000,14.62
2,2024-11-01 00:00:00.000000,4329473,"40.7578106,-73.996801 40.7604506,-74.003221 40...",Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,29.20,191.000000,36061NY09929,Midtown-DOT,40.755082,-73.990415,N Side of W 39th between 8th and 7th Ave,Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,0.006945,641646,2024-11-01 02:00:00.000,13.09
3,2024-11-01 00:00:00.000000,4329473,"40.7578106,-73.996801 40.7604506,-74.003221 40...",Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,29.20,191.000000,36061NY09929,Midtown-DOT,40.755082,-73.990415,N Side of W 39th between 8th and 7th Ave,Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,0.006945,641667,2024-11-01 03:00:00.000,12.17
4,2024-11-01 00:00:00.000000,4329473,"40.7578106,-73.996801 40.7604506,-74.003221 40...",Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,29.20,191.000000,36061NY09929,Midtown-DOT,40.755082,-73.990415,N Side of W 39th between 8th and 7th Ave,Manhattan,LINCOLN TUNNEL E CENTER TUBE NJ - NY,0.006945,641688,2024-11-01 04:00:00.000,11.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4802623,2024-11-26 17:00:00.000000,4329508,"40.75766,-73.99687 40.7604,-74.00328 40.76197,...",Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,6.83,811.666667,36061NY09734,Broadway/35th St,40.750690,-73.987830,Broadway ES1N of 35th St,Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,0.011415,656361,2024-11-30 19:00:00.000,6.09
4802624,2024-11-26 17:00:00.000000,4329508,"40.75766,-73.99687 40.7604,-74.00328 40.76197,...",Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,6.83,811.666667,36061NY09734,Broadway/35th St,40.750690,-73.987830,Broadway ES1N of 35th St,Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,0.011415,656383,2024-11-30 20:00:00.000,5.45
4802625,2024-11-26 17:00:00.000000,4329508,"40.75766,-73.99687 40.7604,-74.00328 40.76197,...",Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,6.83,811.666667,36061NY09734,Broadway/35th St,40.750690,-73.987830,Broadway ES1N of 35th St,Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,0.011415,656404,2024-11-30 21:00:00.000,5.58
4802626,2024-11-26 17:00:00.000000,4329508,"40.75766,-73.99687 40.7604,-74.00328 40.76197,...",Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,6.83,811.666667,36061NY09734,Broadway/35th St,40.750690,-73.987830,Broadway ES1N of 35th St,Manhattan,LINCOLN TUNNEL W CENTER TUBE NY - NJ,0.011415,656425,2024-11-30 22:00:00.000,5.97
